In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import requests
import folium
import datetime
import plotly.express as px
import plotly.graph_objects as go
from tqdm.notebook import tqdm
import time
import json
import re

# Изменения год к году

# Падение перелетов из России:

Основная гипотеза, которую я хочу проверить: Россия в результате событий 2022 и 2023 годов оказалась отрезана от мировых маршрутов авиаперевозок.

Для начала воспользуемся API Opensky network - сайта, предоставляющего информацию об авиарейсах. Однако, проблема этого источника в том, что он трекает совсем не все страны. Например, он не умеет трекать перелеты в Китае, почти всей Африке, Азии и Южной Америке. То есть там есть довольно точные данные о перелетах в Европе и Северной Америке, а об остальных регионах информации совсем мало. Однако, преимуществом данной апи является то, что данные тут можно получить очем быстро. Поэтому, чтобы дать причину делать гипотезу, которую я сделал, давайте посмотрим на количество перелетов из Шереметьево, главного аэропорта России, в разные места за неделю 4 июня-11 июня за 2023, 2022 и 2021 года. (Opensky разрешает получать данные о конкретном аэропорте не более чем за неделю)

In [3]:
username = 'Stankevich'
password = 'vonmi9-ruHcig-pabdij'

In [4]:
response = requests.get('https://opensky-network.org/api')

In [102]:
start_2023 = datetime.datetime(2023, 6, 1, 0, 0)
end_2023 = datetime.datetime(2023, 6, 8, 0, 0)

start_2023 = int(datetime.datetime.timestamp(start_2023))
end_2023 = int(datetime.datetime.timestamp(end_2023))

response = requests.get(f'https://opensky-network.org/api/flights/departure?airport=UUEE&begin={start_2023}&end={end_2023}')
print('Столько перелетов из Шереметьево за прошедшую неделю 2023 года', len(response.json()))

JSONDecodeError: ignored

In [101]:
start_2022 = datetime.datetime(2022, 6, 1, 0, 0)
end_2022 = datetime.datetime(2022, 6, 8, 0, 0)

start_2022 = int(datetime.datetime.timestamp(start_2022))
end_2022 = int(datetime.datetime.timestamp(end_2022))

response = requests.get(f'https://opensky-network.org/api/flights/departure?airport=UUEE&begin={start_2022}&end={end_2022}')
print('Столько перелетов из Шереметьево за прошедшую неделю 2022 года', len(response.json()))

In [ ]:
start_2021 = datetime.datetime(2021, 6, 1, 0, 0)
end_2021 = datetime.datetime(2021, 6, 8, 0, 0)

start_2021 = int(datetime.datetime.timestamp(start_2021))
end_2021 = int(datetime.datetime.timestamp(end_2021))

response = requests.get(f'https://opensky-network.org/api/flights/departure?airport=UUEE&begin={start_2021}&end={end_2021}')
print('Столько перелетов из Шереметьево за прошедшую неделю 2021 года', len(response.json()))

Как видим, количество перелетов снизилось в разы за послдение 2 года. Но это лишь перелеты в ограниченное число стран, тут, например, нет Китая. Так что для более тщательного анализа попробуем поискать данные обо всем мире.

# Анализ данных FlightRadar24

Для оценки связи России с остальным миром авиапутями, сначала получим информацию обо всех перелетах в мире за неделю. Нам будет важно лишь количество самолетов, выполняющих рейсы по заданным маршрутам. Потом будем смотреть, получится ли выделить Россию в отдельный кластер и на сколько сильно Россия связана с другими странами.

Будем качать данные с флайт радара. Система там муторная в том плане, что после каждого запроса в аэропорт мы можем лишь узнать, в какие другие аэропорты оттуда есть перелеты. Далее же нам нужно делать запрос в конкретный маршрут из этого аэропорта в его "соседа", и только тогда можно будет получить данные о том, сколько самолетов летает по данному маршруту.

*1) Получим все аэропорты, которые доступны на FlighRadar24:*

In [6]:
s = requests.session()

In [7]:
#Все возможные аэропорты:
headers = {'User-Agent': 'Mac'}
r = s.get('https://www.flightradar24.com/data/airports/---', headers=headers)
all_airports = re.findall(r'\<small\>\(([A-Z]{3})/', r.text)

### Функции для скачивания данных:

Скачивание данных - это был самый сложный процесс в проекте, на который ушло более 30 часов. FlightRadar очень сильно ограничивает количество запросов, поэтому пришлось разделить весь список аэропортов на буквы (всего 26) и отдельно сохранять по частям накопленную информацию.

*1.5) Разделим на группы:*

In [28]:
alphabet = 'qwertyuiopasdfghjklzxcvbnm'.upper()

airport_groups = {}

for sym in alphabet:
    airport_groups[sym] = []
    for i in all_airports:
        if i[0] == sym.upper():
            airport_groups[sym].append(i)
        
#Разбиваем на группки

In [32]:
#Пример для A
airport_groups['A'][:10]

['AAH', 'AAL', 'AAR', 'ABD', 'ABA', 'ABZ', 'ABR', 'AHB', 'ABJ', 'ABI']

*2) Функция для нахождения доступных направлений из данного аэропорта:*

In [22]:
def get_airports_destinations(iata):
    """
    iata: str - IATA of the airport
    return: np.array - neighbour_iatas
    """
    time.sleep(1)
    headers = {"User-Agent": "Mac"}
    r = s.get(f'https://www.flightradar24.com/data/airports/{iata}/routes', headers=headers)
    
    #Таймлимит
    if re.findall('Just a moment', r.text):
        raise Exception('These capitalists in FlightRadar24.')
    
    #Нет рейсов
    try:
        r = json.loads(r.text.split("arrRoutes=")[-1].split(", arrDates=")[0])
    except:
        return np.array([['NO', 'NO']])

    neighbour_iatas = []
    for airport in r:
        neighbour_iatas.append((airport['iata'], airport['country'].title()))

    return np.array(neighbour_iatas)

In [24]:
#Пример для Шереметьево
svo = get_airports_destinations('SVO')
svo[:10]

array([['KBL', 'Afghanistan'],
       ['ALG', 'Algeria'],
       ['EVN', 'Armenia'],
       ['VIE', 'Austria'],
       ['GYD', 'Azerbaijan'],
       ['GNJ', 'Azerbaijan'],
       ['MSQ', 'Belarus'],
       ['BRU', 'Belgium'],
       ['PEK', 'China'],
       ['PKX', 'China']], dtype='<U20')

*3) Функция для нахождения количества перелетов за неделю по всем направлениям для данного аэропорта:* 

In [25]:
def get_number_of_flights(iata, neighbour_iatas):
    """
    iata: str - IATA of the airport (departure)
    neighbour_iatas: np.array - neighbour_iatas (arrivals)
    return: np.array - number of flights per week to arrival airport from departure airport
    """
    if neighbour_iatas[0,0] == 'NO' and neighbour_iatas[0,1] == 'NO':
        return [0]

    cookie = r.cookies.get_dict()
    headers = {"User-Agent": "Mac",
                "Content-Type": "application/json",
                "x-fetch": "true"}



    number_of_flights = []
    for neig, country in neighbour_iatas:
        s = requests.session()
        time.sleep(0.63)
        
        #На пустой респонс/не существует респонса
        try:
            response = s.get(f'https://www.flightradar24.com/data/airports/{iata.lower()}/routes?get-airport-arr-dep={neig.lower()}',
                             cookies=cookie, headers=headers).json()
            if response == []:
                number_of_flights.append(0)
                continue
        except:
            number_of_flights.append(0)
            continue
        

            
        #На таймлимит
        try:
            if response['error']:
                raise Exception('These capitalists in FlightRadar24.')
        except:
            pass


        #Если есть только departues
        try:
            if response['arrivals']:
                routes = response['arrivals'][country]['airports'][neig]['flights'].keys()
                n = 0
                for route in routes:
                    n += len(response['arrivals'][country]['airports'][neig]['flights'][route]['utc'])

                number_of_flights.append(n)
        except:
            number_of_flights.append(0)
                    

    return np.array(number_of_flights)

In [27]:
#Пример для Шереметьево
svo_flights = get_number_of_flights('SVO', svo)

In [18]:
svo_flights

array([  8,   4,   8,   8,  32,   8,   1,  48,  17,  16,   8,  29,  32,
        88,  91,   8,  32,  16,   3,  33,  10,  43,  56,  52,  24,  16,
        48,   8,  41,  16,  24,  47,   8,   8,  48,  17,   3,   3,   5,
        74,   6,  24, 166, 237,  32,  24,  20,  18,  49,  59,   6,   4,
        24,  42,  37, 101,  16])

*3) Функция для сохрананения информации обо всех аэропортах по букве:*

In [33]:
def get_dataframe(letter, cooldown = True):
    
    for iata in airport_groups[letter]:
        print(iata)
        
        #Если все норм
        try:
            destinations = get_airports_destinations(iata)
            flights = get_number_of_flights(iata, destinations)
            lines = []
            
            for i in range(len(destinations)):
                lines.append((iata, destinations[i][0], flights[i]))
                
        #Если банит     
        except:
            time.sleep(70)
            
            destinations = get_airports_destinations(iata)
            flights = get_number_of_flights(iata, destinations)
            lines = []
            
            for i in range(len(destinations)):
                lines.append((iata, destinations[i][0], flights[i]))

        #Сохраняем результат
        globals()[f'df_{iata}'] = pd.DataFrame(lines, columns = ['Departure', 'Arrival', 'Number of flights'])

*4) Функция для сохранения csv файла по одной букве локально*

In [34]:
def save_csv(letter):
    
    df = pd.DataFrame(columns = ['Departure', 'Arrival', 'Number of flights'])

    
    for iata in airport_groups[letter]:
        df = pd.concat([df, globals()[f'df_{iata}']])

        
    df.reset_index(drop = True, inplace=True)
    df.to_csv(f'{letter}_airports.csv', index=False)

После 30 часов пыток получили информацию о перелетах по всему миру за неделю.

### Научимся получать информацию из IATA, ICAO аэропортов.

IATA -  это код аэропрта из 3 букв. ICAO - из 4 букв. На FlightRadar используется IATA. В пакетах, которые мы будем использовать для перевода кода аэропорта в информацию о нем - испольуется ICAO.

In [19]:
! pip install airportsdata

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.3 MB/s eta 0:00:00


Следующий пакет - для перевода кодов стран из 2-буквенных в названия стран.

In [20]:
! pip install country_converter --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.5/44.5 kB 2.4 MB/s eta 0:00:00


Это скачали на гитхабе как пакет с ICAO аэроптортов. Но, к сожалению, тут есть далеко не всё, поэтому скачаем еще одну табличку с аэропортами. Также везде поменяем названия стран с аббривеатур в 2 буквы в нормальное состояние.

In [21]:
import country_converter
import airportsdata
airportscoords = airportsdata.load()

In [36]:
#Создаем словарь для перевода IATA аэропорта в всю интересующую нас информацию
iata_info = {}
for icao in airportscoords.keys():
    iata_info[airportscoords[icao]['iata']] = {'name': airportscoords[icao]['name'],
                                               'city': airportscoords[icao]['city'],
                                               'country': airportscoords[icao]['country'],
                                               'lat': airportscoords[icao]['lat'],
                                               'lon': airportscoords[icao]['lon'],
                                               'tz': airportscoords[icao]['tz']}

#Используем еще один датасет, т.к. данный не знает много аэропортов
iatas = pd.read_csv('iatas.csv')
iatas = iatas.to_dict()

for row in iatas['code'].keys():
    if not (iatas['code'][row] in iata_info.keys()):
        iata_info[iatas['code'][row]] = {'name': iatas['name'][row],
                                         'city': iatas['city_code'][row],
                                         'country': iatas['country_id'][row],
                                         'lon': iatas['location'][row].split()[1][1:],
                                         'lat': iatas['location'][row].split()[2][:-1],
                                         'tz': iatas['time_zone_id'][row]}

#Почему-то в пакете Намибия превратилась в nan, видимо, человеческая ошибка. Меняем это.
for i in iata_info.keys():
  if iata_info[i]['country'] is np.nan:
    iata_info[i]['country'] = 'Namibia'                                         



#Останутся без городов около 60 аэропортов в итоге, среди них почти все имеют 1-2 перелета в неделю.

In [38]:
#все страны для перевода из двоичного названия в нормальное
countrydict ={}

abbrv_countries = []
for i in iata_info.keys():
    abbrv_countries.append(iata_info[i]['country'])

for abbrv in set(abbrv_countries):
    countrydict[abbrv] = country_converter.convert(abbrv, to = 'name_short')



for iata in iata_info.keys():
    iata_info[iata]['country'] = countrydict[iata_info[iata]['country']]

Вот пример того, что у нас получилось:

In [40]:
iata_info['SVO']

{'name': 'Sheremetyevo International Airport',
 'city': 'Moscow',
 'country': 'Russia',
 'lat': 55.9725990295,
 'lon': 37.4146003723,
 'tz': 'Europe/Moscow'}

### Работа с данными

Импортируем скачанные csv таблички и строим датафреймы из них:

In [41]:
import os
import glob

In [ ]:
!unzip /content/Airportcsvs.zip

In [43]:
data = pd.DataFrame()

for file in glob.glob('/content/Airportcsvs/*.csv'):
    data = pd.concat([data, pd.read_csv(file)])

data.reset_index(drop=True, inplace=True)

Удалим те маршруты, где нет перелетов на самом деле в течение недели и посмотрим на то, что у нас получилось.

In [44]:
data = data[data['Number of flights'] != 0]
print(data.shape)
data.sample(5)

(59323, 3)


,Departure,Arrival,Number of flights
30461,OMA,SFB,2
13813,SMI,SKG,8
44289,DPA,YHM,1
52405,NTG,KWL,4
22263,BBK,GBE,3


---

Добавим полученные данные в датасет с маршрутами:

In [45]:
data = data[data['Departure'].isin(iata_info.keys()) & data['Arrival'].isin(iata_info.keys())]

data['Departure_city'] = data['Departure'].apply(lambda x: iata_info[x]['city'])
data['Arrival_city'] = data['Arrival'].apply(lambda x: iata_info[x]['city'])

data['Departure_country'] = data['Departure'].apply(lambda x: iata_info[x]['country'])
data['Arrival_country'] = data['Arrival'].apply(lambda x: iata_info[x]['country'])

data['Departure_airport'] = data['Departure'].apply(lambda x: iata_info[x]['name'])
data['Arrival_airport'] = data['Arrival'].apply(lambda x: iata_info[x]['name'])

data['Departure_lat'] = data['Departure'].apply(lambda x: iata_info[x]['lat'])
data['Arrival_lat'] = data['Arrival'].apply(lambda x: iata_info[x]['lat'])

data['Departure_lon'] = data['Departure'].apply(lambda x: iata_info[x]['lon'])
data['Arrival_lon'] = data['Arrival'].apply(lambda x: iata_info[x]['lon'])

data['Departure_tz'] = data['Departure'].apply(lambda x: iata_info[x]['tz'])
data['Arrival_tz'] = data['Arrival'].apply(lambda x: iata_info[x]['tz'])

data.sample(5)

<ipython-input-45-ae5b14ed9de4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Departure_city'] = data['Departure'].apply(lambda x: iata_info[x]['city'])


,Departure,Arrival,Number of flights,Departure_city,Arrival_city,Departure_country,Arrival_country,Departure_airport,Arrival_airport,Departure_lat,Arrival_lat,Departure_lon,Arrival_lon,Departure_tz,Arrival_tz
4611,AGP,CWL,6,Malaga,Cardiff,Spain,United Kingdom,Malaga Airport,Cardiff International Airport,36.6749,51.396702,-4.49911,-3.34333,Europe/Madrid,Europe/London
17099,SUV,LKB,1,Nausori,Lakeba Island,Fiji,Fiji,Nausori International Airport,Lakeba Island Airport,-18.043301,-18.1992,178.559006,-178.817001,Pacific/Fiji,Pacific/Fiji
52976,NCE,MLA,2,Nice,Luqa,France,Malta,Nice-Cote d'Azur Airport,Luqa Airport,43.658401,35.857498,7.21587,14.4775,Europe/Paris,Europe/Malta
13738,SSA,SDU,44,Salvador,Rio De Janeiro,Brazil,Brazil,Deputado Luiz Eduardo Magalhaes International ...,Santos Dumont Airport,-12.908611,-22.9105,-38.322498,-43.163101,America/Bahia,America/Sao_Paulo
29865,ORF,CMH,2,Norfolk,Columbus,United States,United States,Norfolk International Airport,John Glenn Columbus International Airport,36.894604,39.996947,-76.201229,-82.892159,America/New_York,America/New_York


---

Можем визуализировать один из аэропортов. Посмотрим, куда летают самолеты из аэропорта Стамбула IST

In [46]:
ist = data[data['Departure'] == 'IST']

In [48]:
#Точки, куда прилетает
fig = px.scatter_geo(ist,
                     lat = 'Arrival_lat',
                     lon = 'Arrival_lon',
                     hover_name='Arrival_city',
                     projection='orthographic',
                     size = 'Number of flights')



#Маршруты
for i in range(len(ist)):
    fig.add_trace(
        go.Scattergeo(
            hoverinfo = 'text',
            text = ist['Arrival_city'].iloc[i],
            lon = [ist['Arrival_lon'].iloc[i], ist['Departure_lon'].iloc[i]],
            lat = [ist['Arrival_lat'].iloc[i], ist['Departure_lat'].iloc[i]],
            mode = 'lines',
            line = dict(width = 0.1, color = 'blue')
        )
    )



#Стамбул
fig.add_scattergeo(lat = ist['Departure_lat'],
                   lon = ist['Departure_lon'],
                   hovertext = ist['Departure_city'])



fig.update_layout(margin=dict(l=3, r=3, t=0, b=0),
                  showlegend = False)
fig.show()

### Проанализируем как вообще обстоят дела в полетах, поищем интересные факты.

Сначала посмотрим, какие аэропорты вообще имеют больше всего перелетов.

In [49]:
total_flights_by_airports = (data.groupby('Arrival')['Number of flights'].sum() + 
                           data.groupby('Departure')['Number of flights'].sum())

total_flights_by_airports.fillna(data.groupby('Arrival')['Number of flights'].sum(), inplace = True)
total_flights_by_airports.fillna(data.groupby('Departure')['Number of flights'].sum(), inplace = True)

In [50]:
fig = px.bar(total_flights_by_airports.sort_values()[-15:],
             orientation='v',
             height = 500,
             title = 'Самые загруженные города мира')
fig.show()

Еще посмотрим на то, какие города больше всего загружены:

In [51]:
total_flights = pd.DataFrame(columns = ['iata','airport', 'lat', 'lon', 'number of flights', 'city', 'country'])

total_flights['iata'] = total_flights_by_airports.index
total_flights['number of flights'] = total_flights_by_airports.values
total_flights['airport'] = total_flights['iata'].apply(lambda x: iata_info[x]['name'])
total_flights['lat'] = total_flights['iata'].apply(lambda x: iata_info[x]['lat'])
total_flights['lon'] = total_flights['iata'].apply(lambda x: iata_info[x]['lon'])
total_flights['city'] = total_flights['iata'].apply(lambda x: iata_info[x]['city'])
total_flights['country'] = total_flights['iata'].apply(lambda x: iata_info[x]['country'])
total_flights['city'].loc[total_flights['city'] == ''] = total_flights[total_flights['city'] == '']['airport']

<ipython-input-51-af0649d4b922>:10: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [52]:
total_flights_by_city = total_flights.drop_duplicates(subset = ['city',	'country'])[['city','country','lat','lon']].merge(
                                                      total_flights.groupby(['country', 'city'], as_index=False)['number of flights'].sum())


Сгруппировали все по городам, теперь посмотрим на то, какие города самые нагруженные по общему количеству перелетов.

In [53]:
fig = px.scatter_geo(total_flights_by_city,
                     hover_name='city',
                     lat='lat',
                     lon = 'lon',
                     color = 'country',
                     size = 'number of flights'
                     )

fig.update_layout(margin=dict(l=3, r=3, t=0, b=0),
                  showlegend = False,
                  geo = dict(scope = 'usa'))

fig.show()

In [54]:
total_flights_by_city

,city,country,lat,lon,number of flights
0,Anaa Airport,French Polynesia,-17.3526,-145.509995,1.0
1,AAD,Somalia,6.09628635,46.637708371259194,1.0
2,Annabah,Algeria,36.822201,7.80917,123.0
3,Apalachicola,United States,29.727549,-85.027378,1.0
4,Buariki,Kiribati,0.185278,173.636993,4.0
...,...,...,...,...,...
4347,Wollaston Lake,Canada,58.106899,-103.171997,14.0
4348,Zunyi,China,27.5895,107.0007,379.0
4349,Sylhet,Bangladesh,24.9632,91.866798,222.0
4350,Tymovskoye,Russia,50.669201,142.761002,4.0


In [55]:
total_flights_by_city.sort_values('number of flights', inplace=True)

fig = px.bar(total_flights_by_city[-25:], x = 'number of flights', y = 'city',
             orientation='h',
             color = 'country',
             height = 700,
             title = 'Самые загруженные города мира')
fig.show()

Также, интересно посмотреть на самые загруженные авиамаршруты мира

In [57]:
most_busy_routes = pd.DataFrame()
most_busy_routes['Route'] = data['Departure_city'] + '-' + data['Arrival_city']
most_busy_routes['Number of flights'] = (data + data.rename(columns = {'Departure_city':'Arrival_city', 'Arrival_city':'Departure_city'}))['Number of flights']
most_busy_routes.sort_values('Number of flights', inplace = True)

In [58]:
fig = px.bar(most_busy_routes[-30:], y = 'Number of flights', x = 'Route',
             orientation='v',
             height = 700,
             title = 'Самые загруженные маршруты мира')
fig.show()

---

### Кластеризация

Теперь давайте составим граф, вершинами которого будут являться города, а ребрами будут перелеты. Соответсвенно, веса ребер - количество перелетов между городами. Хотим попробовать "выцепить" Россию в отдельном кластере при проделывании процедуры спектральной кластеризации.

In [59]:
! pip install networkx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [60]:
import networkx as nx

In [61]:
#множество вершин:
vertexes = set([*data['Departure'], *data['Arrival']])
len(vertexes)

4489

Создаем Лапласиан графа:

In [62]:
G = nx.MultiGraph()

#all cities
G.add_nodes_from(vertexes)

#all flights
e = list(zip(data['Arrival'], data['Departure'], data['Number of flights']))

G.add_weighted_edges_from(e, weight = 'flights') 
L = nx.laplacian_matrix(G)

---

In [63]:
from sklearn.base import ClusterMixin
from sklearn.cluster import KMeans


class GraphClustering(ClusterMixin):
    def __init__(self, n_clusters=8, n_components=None, **kwargs):
        if n_components is None:
            n_components = n_clusters

        self.n_components = n_components
        self.kmeans = KMeans(n_clusters=n_clusters, **kwargs)

    def fit_predict(self, L, y=None):
        eigenvectors = self._generate_eigenvectors(L)
        labels = self.kmeans.fit_predict(eigenvectors[:, 1:])
        return labels

    def _generate_eigenvectors(self, L):
        if self.n_components is None:
          return np.linalg.eigh(L)[1]
        else:
          return np.linalg.eigh(L)[1][:,:self.n_components]

Зафитим модель, потом добавим лейблы в датафрейм и визуализируем. Если выбирать слишком маленькое число кластеров, то будут отваливаться маленькие зоны, мы же хотим чтобы появились большие кластеры, поэтому выберем n_clusters 180.

In [68]:
model = GraphClustering(n_clusters=180)
labels = model.fit_predict(L.todense())

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [72]:
airports = pd.DataFrame()

airports['label'] = labels
airports['airport'] = list(vertexes)
airports['lat'] = airports['airport'].apply(lambda x: iata_info[x]['lat'])
airports['lon'] = airports['airport'].apply(lambda x: iata_info[x]['lon'])
airports['number of flights'] = airports['airport'].apply(lambda x: G.degree(weight = 'flights')[x])
airports['country'] = airports['airport'].apply(lambda x: iata_info[x]['country'])

In [70]:
airports.sample(5)

,label,airport,lat,lon,number of flights,country
3621,0,BWA,27.505699,83.416298,280,Nepal
199,0,YTW,36.80990525,81.79149269333863,69,China
2363,170,TPQ,21.4195,-104.843002,48,Mexico
1161,167,BWB,-20.864401,115.405998,2,Australia
4079,0,VLI,-17.699301,168.320007,124,Vanuatu


In [71]:
fig = px.scatter_geo(airports,
                     hover_name='airport',
                     lat='lat',
                     lon = 'lon',
                     color = 'label'
                     )

fig.update_layout(margin=dict(l=3, r=3, t=0, b=0),
                  showlegend = False)

fig.show()

Перезапускал алгоритм несколько раз. Иногда Северная и Южная Америки отделялись от кластера Евразия-Африка, когда-то же отделялась отдельно Южная Америка, Северная Америка и Евразия-Африка. Но абсолютно всегда оставались аномальные зоны, в которых явно выделялись отдельные кластера: это северные территории такие как Аляска, Север Норвегии, Гудзонов залив, Северо-Восточная Сибирь, а также отделялись Западная Австралия, Кения и Иран. Понятно, что также туда попадают всякие единичные отдаленные аэропорты или группы аэропортов, но я выделил самые постоянные и большие.

In [73]:
fig = px.scatter_geo(airports,
                     hover_name='airport',
                     lat='lat',
                     lon = 'lon',
                     color = 'label',
                     size = 'number of flights',
                     #projection='orthographic'
                     )

fig.update_layout(margin=dict(l=3, r=3, t=0, b=0),
                  showlegend = False, geo = dict(scope = 'south america'))

fig.show()

То, почему Южная Америка выделилась в отдельный кластер можно объяснить тем, что в ней очень много перелетов внутренних, как мы это увидим дальше. В общем число перелетов здесь выше, чем в среднем в Азии, Океании или Африке. Тут сильны внутренние перелеты, в итоге образуется отдельный кластер.

---

### Исследуем связи между странами

Для того, чтобы понять, насколько созависимы страны, будем использовать normalized_cut_size графа. По сути, результат будет показывать, насколько сильны связи между двумя странами, учитывая общее число связей каждой из стран. Для примера посмотрим Беларусь, Россию, Иран (который у нас выделился в отдельный кластер в прошлом разделе) и Чехию.

In [74]:
codependence = pd.DataFrame(columns = ['country', 'codependence'])
graph_countries = airports['country']

for i in graph_countries.unique():
    S = np.array(G.nodes)[graph_countries == 'Belarus']
    T = np.array(G.nodes)[graph_countries == i]

    if nx.normalized_cut_size(G, S, T) !=0:
        codependence = pd.concat([codependence, pd.DataFrame(columns = ['country', 'codependence'],
                                                             data = [[i, nx.normalized_cut_size(G, S, T, weight = 'flights')]])])



codependence.sort_values(by = 'codependence', inplace = True)
fig = px.bar(codependence, y = 'codependence', x = 'country', orientation='v',
             color = 'codependence',
             height = 500,
             title = 'Созависимость с Беларусью')
fig.show()

Как-то грустно. Очень большая созависимость с Россией, в то время как с другими странами перелетов совсем немного. А еще тут нет самой Беларуси, то есть там нет внутренних перелетов. Оказывается, в Беларуси есть только один аэропорт в Минске и, соответственно, внутренних перелетов нет.

In [75]:
codependence = pd.DataFrame(columns = ['country', 'codependence'])
graph_countries = airports['country']

for i in graph_countries.unique():
    S = np.array(G.nodes)[graph_countries == 'Russia']
    T = np.array(G.nodes)[graph_countries == i]

    if nx.normalized_cut_size(G, S, T) !=0:
        codependence = pd.concat([codependence, pd.DataFrame(columns = ['country', 'codependence'],
                                                             data = [[i, nx.normalized_cut_size(G, S, T, weight = 'flights')]])])



codependence = codependence.sort_values(by = 'codependence').iloc[25:]
fig = px.bar(codependence, y = 'codependence', x = 'country', orientation='v',
             color = 'codependence',
             height = 500,
             title = 'Созависимость с Россией')
fig.show()

Плюс-минус ожидаемый результат. Сильные связи с постсоветским пространством, с остальными странами связи уже скорее слабые.

In [76]:
codependence = pd.DataFrame(columns = ['country', 'codependence'])
graph_countries = airports['country']

for i in graph_countries.unique():
    S = np.array(G.nodes)[graph_countries == 'Iran']
    T = np.array(G.nodes)[graph_countries == i]

    if nx.normalized_cut_size(G, S, T) !=0:
        codependence = pd.concat([codependence, pd.DataFrame(columns = ['country', 'codependence'],
                                                             data = [[i, nx.normalized_cut_size(G, S, T, weight = 'flights')]])])



codependence = codependence.sort_values(by = 'codependence')[10:]
fig = px.bar(codependence, y = 'codependence', x = 'country', orientation='v',
             color = 'codependence',
             height = 500,
             title = 'Созависимость с Ираном')
fig.show()

Тут прям совсем все грустно. Не ожидал, что результат будет сильно хуже Беларуси, но вот... В Иране просто почти все перелеты внутренние, а другие страны туда особо не летают.

In [77]:
codependence = pd.DataFrame(columns = ['country', 'codependence'])
graph_countries = airports['country']

for i in graph_countries.unique():
    S = np.array(G.nodes)[graph_countries == 'France']
    T = np.array(G.nodes)[graph_countries == i]

    if nx.normalized_cut_size(G, S, T) !=0:
        codependence = pd.concat([codependence, pd.DataFrame(columns = ['country', 'codependence'],
                                                             data = [[i, nx.normalized_cut_size(G, S, T, weight = 'flights')]])])



codependence = codependence.sort_values(by = 'codependence')[95:]
fig = px.bar(codependence, y = 'codependence', x = 'country', orientation='v',
             color = 'codependence',
             height = 700,
             title = 'Созависимость с Францией')
fig.show()

Франция тут выглядит как диверсифицированное государство в плане перелетов. Нет сильной зависимости от внутренних перелетов или от какого-то другого государства.

---

# Итог

По результатам анализа мировых перелетов мы не можем сделать вывод о том, что Россия осталась отрезана от мировых авиамаршрутов. Все еще сохраняются связи с постсоветским пространством, а также с Турцией. Однако, из резреза графа видно, что внешних связей у России на данный момент все равно мало.